# TP2: Planificación de Conexión de Oficinas de Atención a Internet

Integrantes: Micaela Oliva, Camila Bernardez, Sol Valeggia, Juan Castore

## Contexto
Estamos en el área de planificación de una empresa de cobros minoristas, y tenemos la responsabilidad de conectar a Internet 56 oficinas de atención al cliente, distribuidas en el micro y macrocentro de la ciudad. La empresa de telefonía nos ha propuesto hasta 10 centrales operativas que pueden ser utilizadas para abastecer estas oficinas, sujetas a las siguientes restricciones:

### Restricciones
1. Cada oficina debe estar conectada con una central operativa.
2. Cada oficina tiene una demanda de operaciones por hora, y cada central operativa tiene una capacidad máxima de 15.000 operaciones por hora. Esto implica que la suma de las operaciones por hora de las oficinas conectadas a cada central no puede superar las 15.000 operaciones.

## Objetivo
El objetivo de la planificación es determinar:
1. Qué centrales operativas utilizar.
2. Qué central operativa debe atender a cada oficina.

Para cumplir con estas restricciones y minimizar el costo total, considerando:

- **Costo de apertura**: USD 5.700 por cada central operativa utilizada.
- **Costo de cableado**: Cada oficina se conecta a una central mediante un cable directo cuya longitud es $d_{ij}$ en metros entre la oficina $i $ y la central $j$. El cable tiene un costo de USD 17 por cada 1.000 metros.


### 1. Modelo de Programación Lineal Entera
   - **Variables de Decisión**: 
     - $x_{ij}$: Variable binaria que indica si la oficina $i$ está conectada a la central $j$.
     - $y_j$: Variable binaria que indica si la central $j$ está en operación.
   - **Función Objetivo**: Minimizar el costo total, que se compone del costo de apertura de las centrales y el costo de cableado.
   - **Restricciones**: 
     - Cada oficina debe estar conectada a una y solo una central operativa.
     - La suma de operaciones de las oficinas conectadas a una central no puede exceder su capacidad máxima.
     - Solo pueden conectarse oficinas a centrales que estén en operación.



### 2. Análisis de Restricciones Adicionales
   - Implementar una nueva restricción en el modelo donde el número de oficinas conectadas a cada central no supere las 10.
   - Analizar el impacto de esta restricción en los resultados del modelo, especialmente en el costo total y en la distribución de las conexiones.


### 3. Determinación de la Capacidad Mínima
   - Incrementar gradualmente la capacidad máxima permitida de operaciones por hora para cada central y evaluar la factibilidad del modelo.
   - Determinar la capacidad mínima que permite una asignación factible para todas las oficinas.


### 4. Evaluación del Tiempo de Resolución
   - Generar instancias aleatorias variando el número de oficinas y centrales operativas.
   - Medir el tiempo de resolución del solver y determinar el tamaño de instancia que marca el límite de tiempos de resolución aceptables.


## Conclusiones
Este análisis permitirá tomar decisiones informadas sobre la selección y configuración de las centrales operativas y la conexión de las oficinas, optimizando tanto el costo como la eficiencia operativa en el abastecimiento de las oficinas con servicio de Internet.
